In [1]:
import numpy as np
import pandas as pd
import folium
import branca
from folium import plugins
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import geojsoncontour
import scipy as sp
import scipy.ndimage
import base64
from folium.features import DivIcon

In [12]:
def color_producer1(el):
    if el < 0.2:
        return "#FFFFFF"
    elif 0.2 <= el < 0.4:
        return "#BBBBFF"
    elif 0.4 <= el < 0.6:
        return "#8888FF"
    elif 0.6 <= el < 0.8:
        return "#4444FF"

    else:
        return "#0000FF"

def color_producer2(el):
    if el < 0.2:
        return "#FFFFFF"
    elif 0.2 <= el < 0.4:
        return "#FFBBBB"
    elif 0.4 <= el < 0.6:
        return "#FF8888"
    elif 0.6 <= el < 0.8:
        return "#FF4444"

    else:
        return "#FF0000"
def add_layer(df,text,cc): 
    if cc == 1:
        colors = ["#FFFFFF", "#BBBBFF", "#8888FF", "#4444FF", "#0000FF"]
    elif cc == 2:
        colors = ["#FFFFFF", "#FFBBBB", "#FF8888", "#FF4444", "#FF0000"] 
    f1 = folium.FeatureGroup(name='{}'.format(text))
 
    x_orig = np.asarray(df.long.tolist())
    y_orig = np.asarray(df.lat.tolist())
    z_orig = np.asarray(df.EI.tolist())

    x_arr          = np.linspace(np.min(x_orig), np.max(x_orig), 1000)
    y_arr          = np.linspace(np.min(y_orig), np.max(y_orig), 1000)
    x_mesh, y_mesh = np.meshgrid(x_arr, y_arr)

    z_mesh = griddata((x_orig, y_orig), z_orig, (x_mesh, y_mesh), method='linear')

    sigma = [5, 5]
    z_mesh = sp.ndimage.filters.gaussian_filter(z_mesh, sigma, mode='constant')

    contourf = plt.contourf(x_mesh, y_mesh, z_mesh, levels, alpha=0.5, colors=colors, linestyles='None', vmin=vmin, vmax=vmax)

    geojson = geojsoncontour.contourf_to_geojson(
        contourf=contourf,
        min_angle_deg=3.0,
        ndigits=5,
        stroke_width=1,
        fill_opacity=0.5)
    geoj = folium.GeoJson(
        geojson,
        style_function=lambda x: {
            'color':     x['properties']['stroke'],
            'weight':    x['properties']['stroke-width'],
            'fillColor': x['properties']['fill'],
            'opacity':   0.6,
        })
    f1.add_child(geoj)

    return f1

def add_point(df,text,cc):
    html = """
    CWB station name:
    <br><a href="https://www.google.com/search?q=%%22%s%%22" target="_blank">%s</a><br>
    EI value: %.2f 
    """
    
    f2 = folium.FeatureGroup(name='{}'.format(text))
    for lt, ln, ei, nm in zip(df.lat,df.long, df.EI, df.id):
        iframe = folium.IFrame(html=html % (nm,nm,ei), width = 200, height = 100)
        if cc == 1:
            f2.add_child(folium.CircleMarker(location = [lt,ln], 
                                        popup = folium.Popup(iframe),
                                        radius = 4,
                                        color = 'grey',
                                        fill_color=color_producer1(ei),
                                        fill_opacity = 0.7))
        elif cc == 2:
            f2.add_child(folium.CircleMarker(location = [lt,ln], 
                                        popup = folium.Popup(iframe),
                                        radius = 4,
                                        color = 'grey',
                                        fill_color=color_producer2(ei),
                                        fill_opacity = 0.7))
    return f2

def add_text(loc,content,c,fig):
    html = '<img src="data:image/png;base64,{}">'.format
    encoded = base64.b64encode(open("../image/{}.jpg".format(fig), 'rb').read()).decode()

    iframe = folium.IFrame(html(encoded), width=200, height=200)
    popup = folium.Popup(iframe, max_width=1000)

    Text = folium.Marker(location=loc, popup=popup,
                         icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html='<font color="{}"><div style="font-size: 16pt">{}</div></font>'.format(c,content),
            ))
    return Text

In [9]:
sp1, sp2 = 'solenopsis_invicta','achatina_fulica'
df1 = pd.read_csv("../output_csv/{}/2017-04_{}.csv".format(sp1,sp1),index_col=0)
df2 = pd.read_csv("../output_csv/{}/2017-04_{}.csv".format(sp2,sp2),index_col=0)
df1 = df1[df1.long>120]
df1 = df1[df1['id'] != 'C0S730'][df1['id'] != '467620'][df1['id'] != '466950'][df1['id'] !='C0W170']
df2 = df2[df2.long>120]
df2 = df2[df2['id'] != 'C0S730'][df2['id'] != '467620'][df2['id'] != '466950'][df2['id'] !='C0W170']

/Users/nghianc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/Users/nghianc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


In [16]:
geomap = folium.Map([23.75, 121], zoom_start=8, tiles="cartodbpositron")
levels = 5
vmin, vmax = 0,1
f1a = add_layer(df1,"Contour",1)
geomap.add_child(f1a)
f1b = add_point(df1,"Markers",1)
geomap.add_child(f1b)
Text1 = add_text([25,119],"南方小花蝽","red","orius_strigicollis")
geomap.add_child(Text1)

f2a = add_layer(df2,"Contour2",2)
geomap.add_child(f2a)
f2b = add_point(df2,"Markers2",2)
geomap.add_child(f2b)
Text2 = add_text([24,119],"中國梨木蝨","blue","cacopsylla_chinensis")
geomap.add_child(Text2)

plugins.Fullscreen(position='topright', force_separate_button=True).add_to(geomap)
geomap.add_child(folium.LayerControl()) 
#geomap.save('../html/test.html')